In [1]:
# some preprocessing on TACO would be helpful before feeding it to models.

In [2]:
!pip install --upgrade Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 2.1 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2


In [1]:
from PIL import Image, ExifTags
from pycocotools.coco import COCO
from matplotlib.patches import Polygon, Rectangle
from matplotlib.collections import PatchCollection
import colorsys
import random
import pylab

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from tqdm import tqdm

import shutil
import os

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [2]:
%%capture

# Download TACO images from the original author
# !rm -rf /content/*
# !mkdir data
# !wget -O data/annotations.json https://raw.githubusercontent.com/pedropro/TACO/master/data/annotations.json
# !wget https://raw.githubusercontent.com/pedropro/TACO/master/download.py
# !python download.py

# alternatively from gdrive
!gdown 1tktEh58DIWad-3NadT1vIu04D9vNoISB
!unzip TACOdata.zip

In [3]:
dataset_path = './data'
anns_file_path = dataset_path + '/' + 'annotations.json'

with open(anns_file_path, 'r') as f:
    dataset = json.loads(f.read())
imgs = dataset['images']

data_source = COCO(annotation_file=anns_file_path)

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


In [4]:
!mkdir yoloTACO
!rm -rf ./yoloTACO/*
!mkdir yoloTACO/images
!mkdir yoloTACO/labels

In [5]:
dst = "./yoloTACO"
coco = COCO(anns_file_path)

for img in tqdm(imgs):

  # reorganize images
  img_id = img['id']
  img_location = './data/'+img['file_name']
  shutil.copy(img_location, os.path.join(dst+'/images/',str(img_id)+'.jpg'))

  # reorganize annotations
  annotation_ids = coco.getAnnIds(img_id)
  if len(annotation_ids) == 0:
    continue

  img_info = data_source.loadImgs(img_id)[0]    
  height = img_info['height']
  width = img_info['width']

  with open(dst+'/labels/'+str(img_id)+'.txt', mode='w') as fp:
    annotations = coco.loadAnns(annotation_ids)
    lines = ''
    for annotation in annotations:
      label = annotation['category_id']
      box = annotation['bbox'] 
      # some annotations have basically no width / height (extremely small), skip them
      if box[2] < 10 or box[3] < 10:
        continue
      # top_x,top_y,width,height ----> cen_x,cen_y,width,height
      # standardize to 0-1
      box[0] = round((box[0] + box[2] / 2) / width, 6)
      box[1] = round((box[1] + box[3] / 2) / height, 6)
      box[2] = round(box[2] / width, 6)
      box[3] = round(box[3] / height, 6)
      # line: label x_center y_center width height
      lines = lines + str(label)
      for i in box:
          lines += ' ' + str(i)
      lines += '\n'
    fp.writelines(lines)

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!


100%|██████████| 1500/1500 [00:09<00:00, 164.05it/s]


In [6]:
# Obtain Exif orientation tag code
for orientation in ExifTags.TAGS.keys():
    if ExifTags.TAGS[orientation] == 'Orientation':
        break

In [7]:
def orientation_reder(image_filepath,size=False):
  # Load image
  I = Image.open(image_filepath)

  h,w = I.size
  a= 1
  # Load and process image metadata
  if I._getexif():
      exif = I._getexif()
      try: 
        a = exif[orientation]
        if a ==0:
          a=1
      except: pass
  if size==True:
    return a, h, w
  else: return a

ori_list=[]
for i in range(1500):
  ori_list.append([i,orientation_reder('yoloTACO/images/'+str(i)+'.jpg')])

In [8]:
def img_box_visualizer(id):
  image_filepath = 'yoloTACO/images/'+str(id)+'.jpg'
  label_filepath = 'yoloTACO/labels/'+str(id)+'.txt'
  pylab.rcParams['figure.figsize'] = (28,28)
  ####################

  # Load image
  print(image_filepath)
  I = Image.open(image_filepath)
  # Show image
  fig,ax = plt.subplots(1)
  plt.axis('off')
  plt.imshow(I)

  # Load mask ids
  annotations = pd.read_csv(label_filepath,header=None,delimiter=' ')
  W,H = I.size
  # Show annotations
  for _,row in annotations.iterrows():
    [x, y, w, h] = row.tolist()[1:]
    x=round((x-w/2)*W)
    y=round((y-h/2)*H)
    w=round(w*W)
    h=round(h*H)
    rect = Rectangle((x,y),w,h,linewidth=4,edgecolor='red',facecolor='none', alpha=0.7, linestyle = '--')
    ax.add_patch(rect)
  plt.show()

In [9]:
# Some pictures' EXIF orientation is not 1 and that may be a problem
# See a problematic example below:
id = 48   
img_box_visualizer(id)


Output hidden; open in https://colab.research.google.com to view.

In [10]:
!wget https://raw.githubusercontent.com/john-science/exif_delete/master/exif_delete.py

--2022-09-18 20:52:47--  https://raw.githubusercontent.com/john-science/exif_delete/master/exif_delete.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3432 (3.4K) [text/plain]
Saving to: ‘exif_delete.py’

exif_delete.py      100%[===================>]   3.35K  --.-KB/s    in 0s      

2022-09-18 20:52:48 (57.2 MB/s) - ‘exif_delete.py’ saved [3432/3432]



In [11]:
from exif_delete import *

In [12]:
# Rotate pictures and save a clean copy
for id,orientation_ in tqdm([i for i in ori_list if i[1]!=1]):
  image_filepath = 'yoloTACO/images/'+str(id)+'.jpg'
  ####################
  if orientation_!=1:
    # Load image
    exif_delete(image_filepath, image_filepath)

100%|██████████| 575/575 [22:09<00:00,  2.31s/it]


In [13]:
#check if there are orientation that's NOT 1
set([orientation_reder('yoloTACO/images/'+str(i)+'.jpg') for i in range(1500)])

{1}

In [14]:
# Visualize again picture 48, see if correct now
id = 48   
img_box_visualizer(id)

Output hidden; open in https://colab.research.google.com to view.

In [15]:
%%capture
# Save processed images and label to google drive

from google.colab import drive
drive.mount('/gdrive')

!zip -r /gdrive/MyDrive/rotated2.zip /content/yoloTACO/*